<a href="https://colab.research.google.com/github/Molten-Ice/Deep-Learning/blob/dev/Tutorials/GPT_from_scratch_andrej_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hi")

Hi


In [ ]:
with open("foundation.txt", 'r', encoding='utf-8') as f:
  text = f.read()

In [ ]:
print(f"Length of dataset in characters: {len(text)}")

Length of dataset in characters: 1240703


In [ ]:
print(text[:1000])

FOUNDATION 
ISAAC ASIMOV 


Contents 

Introduction 

Part I The Psvchohistorians 

Part II The Encyclopedists 

Part III The Mayors 

Part IV The Traders 

Part V The Merchant Princes 

PART I 

THE PSYCHOHISTORIANS 

i. 

HARI SELDON-... bom In the 1 1,988th year of the Galactic Era; died 12,069. The dates are 
more commonly given In terms of the current Foundational Era as - 79 to the year 1 F.E. Born 
to middle-class parents on Flelicon, Arcturus sector (where his father, In a legend of doubtful 
authenticity, was a tobacco grower in the hydroponic plants of the planet), he early showed 
amazing ability in mathematics. Anecdotes concerning his ability are innumerable, and some 
are contradictory. At the age of two, he is said to have ... 

... Undoubtedly his greatest contributions were in the field of psychohistory. Seldon found the 
field little more than a set of vague axioms; he left it a profound statistical science.... 

... The best existing authority we have for the details

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#%'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ\abcdefghijklmnopqrstuvwxyz—‘’”
84


In [ ]:
# create a mapping from characters to integers
stoi = {i:ch for ch, i in enumerate(chars)}
itos = {ch:i for ch, i in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #encode, take a string, output a list of integers
decode = lambda l: ''.join([itos[n] for n in l])#decode, take a list of integers, output a string

print(encode("Hello world!"))
print(decode(encode("Hello world!")))

[34, 58, 65, 65, 68, 1, 76, 68, 71, 65, 57, 2]
Hello world!


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1240703]) torch.int64
tensor([32, 41, 47, 40, 30, 27, 46, 35, 41, 40,  1,  0, 35, 45, 27, 27, 29,  1,
        27, 45, 35, 39, 41, 48,  1,  0,  0,  0, 29, 68, 67, 73, 58, 67, 73, 72,
         1,  0,  0, 35, 67, 73, 71, 68, 57, 74, 56, 73, 62, 68, 67,  1,  0,  0,
        42, 54, 71, 73,  1, 35,  1, 46, 61, 58,  1, 42, 72, 75, 56, 61, 68, 61,
        62, 72, 73, 68, 71, 62, 54, 67, 72,  1,  0,  0, 42, 54, 71, 73,  1, 35,
        35,  1, 46, 61, 58,  1, 31, 67, 56, 78, 56, 65, 68, 69, 58, 57, 62, 72,
        73, 72,  1,  0,  0, 42, 54, 71, 73,  1, 35, 35, 35,  1, 46, 61, 58,  1,
        39, 54, 78, 68, 71, 72,  1,  0,  0, 42, 54, 71, 73,  1, 35, 48,  1, 46,
        61, 58,  1, 46, 71, 54, 57, 58, 71, 72,  1,  0,  0, 42, 54, 71, 73,  1,
        48,  1, 46, 61, 58,  1, 39, 58, 71, 56, 61, 54, 67, 73,  1, 42, 71, 62,
        67, 56, 58, 72,  1,  0,  0, 42, 27, 44, 46,  1, 35,  1,  0,  0, 46, 34,
        31,  1, 42, 45, 51, 29, 34, 41, 34, 35, 45, 46, 41, 44, 35, 27, 40, 45,
      

In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
print(decode(x.item() for x in train_data[:100]))

FOUNDATION 
ISAAC ASIMOV 


Contents 

Introduction 

Part I The Psvchohistorians 

Part II The Ency


In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([32, 41, 47, 40, 30, 27, 46, 35, 41])

In [ ]:
for t in range(block_size):
  context = train_data[:t+1]
  target = train_data[t+1]
  print(f"when the input is {context} the target is {target}")

when the input is tensor([32]) the target is 41
when the input is tensor([32, 41]) the target is 47
when the input is tensor([32, 41, 47]) the target is 40
when the input is tensor([32, 41, 47, 40]) the target is 30
when the input is tensor([32, 41, 47, 40, 30]) the target is 27
when the input is tensor([32, 41, 47, 40, 30, 27]) the target is 46
when the input is tensor([32, 41, 47, 40, 30, 27, 46]) the target is 35
when the input is tensor([32, 41, 47, 40, 30, 27, 46, 35]) the target is 41


In [ ]:
torch.manual_seed(3)
batch_size = 4
block_size = 8

def get_batch(split="train"):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data)-block_size, (batch_size, ))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch()
xb.shape, yb.shape
xb, yb

(tensor([[54, 67, 57,  1, 73, 74, 71, 67],
         [68, 69, 65, 58,  1, 56, 68, 67],
         [67, 68,  1, 72, 62, 60, 67, 72],
         [ 1, 42, 71, 62, 73, 56, 61, 58]]),
 tensor([[67, 57,  1, 73, 74, 71, 67, 58],
         [69, 65, 58,  1, 56, 68, 67, 73],
         [68,  1, 72, 62, 60, 67, 72,  1],
         [42, 71, 62, 73, 56, 61, 58, 71]]))

In [ ]:
print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

----
when input is [54] the target: 67
when input is [54, 67] the target: 57
when input is [54, 67, 57] the target: 1
when input is [54, 67, 57, 1] the target: 73
when input is [54, 67, 57, 1, 73] the target: 74
when input is [54, 67, 57, 1, 73, 74] the target: 71
when input is [54, 67, 57, 1, 73, 74, 71] the target: 67
when input is [54, 67, 57, 1, 73, 74, 71, 67] the target: 58
when input is [68] the target: 69
when input is [68, 69] the target: 65
when input is [68, 69, 65] the target: 58
when input is [68, 69, 65, 58] the target: 1
when input is [68, 69, 65, 58, 1] the target: 56
when input is [68, 69, 65, 58, 1, 56] the target: 68
when input is [68, 69, 65, 58, 1, 56, 68] the target: 67
when input is [68, 69, 65, 58, 1, 56, 68, 67] the target: 73
when input is [67] the target: 68
when input is [67, 68] the target: 1
when input is [67, 68, 1] the target: 72
when input is [67, 68, 1, 72] the target: 62
when input is [67, 68, 1, 72, 62] the target: 60
when input is [67, 68, 1, 72, 62

In [ ]:
print(xb) # input to the transformer

tensor([[54, 67, 57,  1, 73, 74, 71, 67],
        [68, 69, 65, 58,  1, 56, 68, 67],
        [67, 68,  1, 72, 62, 60, 67, 72],
        [ 1, 42, 71, 62, 73, 56, 61, 58]])


In [ ]:
try:
    import einops
except:
    print(f"einops not installed as required, installing...")
    !pip3 install einops
    import einops

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from einops import rearrange, reduce, repeat
torch.manual_seed(3)

class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()
    #Each token directly reads off the logit for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):

    logits = self.token_embedding_table(idx) # (B, T, C) batch, time, channel
    #[4, 8, 84])
    if targets == None:
      loss = None
    else:
      logits = rearrange(logits, 'B T C -> (B T) C')
      targets = rearrange(targets, 'B T -> (B T)')
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    #idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      #get the predicitions
      logits, loss = self(idx)

      #Focus on last time step
      logits = logits[:, -1, :] # becomes (B, C)
      #apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      #sample from the distribution
      
      idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to running sequence
      idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx


m = BigramLanguageModel()
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype = torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 84])
tensor(4.9284, grad_fn=<NllLossBackward0>)

av3/—cT'A68XN/LE%L‘‘kkXeW)5d;ZE1s”-,zM8WJMh-GIVZ 
.””TY4f9nRG4kGd—kCy;p%9*/9yB
DaFwiR7sH64e7QLWgV,,H


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size=32
for steps in range(10000):

  #sample a batch of data
  xb, yb = get_batch('train')

  #evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.3158881664276123


In [ ]:
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


"Youroowis 
" Ke t 
"ORiomor blobyofind yesede -k," mo. akwheerd 

Cher touno Nof tin 
Share sor indes woratow'sto p bed -wanguto l ing st ilce vend id., hen, aldor. ont Pen ghinccandreigherentuisnle 
" ak? icond, wengly his aly Noras l pre, wartis fr stilont ally in irererltre de rine 

f whmsppps 


## The mathematical trick in self-attention

In [ ]:
#toy example
B, T, C = 4, 8, 2
x = torch.rand(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
xbow = torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev=x[b,:t+1] #(t, C)
    xbow[b,t]=torch.mean(xprev, 0)
xbow.shape

torch.Size([4, 8, 2])

In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C)
xbow2.shape, xbow2

(torch.Size([4, 8, 2]), tensor([[[0.3199, 0.9329],
          [0.2211, 0.9348],
          [0.2060, 0.7641],
          [0.2101, 0.7790],
          [0.1774, 0.6962],
          [0.2743, 0.5942],
          [0.2872, 0.5450],
          [0.3209, 0.5052]],
 
         [[0.0414, 0.6998],
          [0.1315, 0.6451],
          [0.3959, 0.6086],
          [0.3549, 0.6772],
          [0.3660, 0.5527],
          [0.4262, 0.5293],
          [0.3883, 0.5443],
          [0.4144, 0.5962]],
 
         [[0.2405, 0.0219],
          [0.2705, 0.0158],
          [0.2658, 0.2286],
          [0.2983, 0.3950],
          [0.3111, 0.3728],
          [0.3794, 0.4181],
          [0.4098, 0.4430],
          [0.3612, 0.4671]],
 
         [[0.9030, 0.9143],
          [0.4728, 0.9273],
          [0.4710, 0.7030],
          [0.4953, 0.7608],
          [0.4975, 0.7072],
          [0.5069, 0.6566],
          [0.5208, 0.6680],
          [0.5357, 0.6292]]]))

In [ ]:
torch.allclose(xbow, xbow2)

True

In [ ]:
xbow[0],xbow2[0]

(tensor([[0.3199, 0.9329],
         [0.2211, 0.9348],
         [0.2060, 0.7641],
         [0.2101, 0.7790],
         [0.1774, 0.6962],
         [0.2743, 0.5942],
         [0.2872, 0.5450],
         [0.3209, 0.5052]]), tensor([[0.3199, 0.9329],
         [0.2211, 0.9348],
         [0.2060, 0.7641],
         [0.2101, 0.7790],
         [0.1774, 0.6962],
         [0.2743, 0.5942],
         [0.2872, 0.5450],
         [0.3209, 0.5052]]))

In [ ]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
print(wei)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


True

In [ ]:
# version 4: self-attention (my attempt)
# afternote, need to linearly project to get q,k,v at the start

torch.manual_seed(3)
B, T, C = 4, 8, 32 #batch, time, channels
x = torch.randn(B, T, C)

tril = torch.tril(torch.ones(T,T))
#Query: What am I looking for
#Key: What do I contain
Q, K, V = torch.randn(B, T, C), torch.randn(B, T, C), torch.randn(B, T, C)
wei = Q @ rearrange(K, 'B T C -> B C T')
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ V
wei.shape, wei[0][:3]

(torch.Size([4, 8, 8]),
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0032, 0.9968, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2699, 0.0012, 0.7289, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]))

In [ ]:
out.shape, out[0][:3]

(torch.Size([4, 8, 16]),
 tensor([[-0.0209, -0.4611,  0.6770, -0.4637, -1.3853,  0.4191,  0.1293,  0.9245,
           0.5460,  0.2836,  0.1412, -0.2600, -0.5766, -0.5271,  0.6131,  0.5250],
         [ 0.1207, -0.0291, -0.1975, -0.1703, -0.5137, -0.2041, -0.3989,  0.5389,
          -0.3234,  0.2389, -0.2635,  0.1992, -0.2563, -0.4836,  0.1775,  0.2862],
         [ 0.7711,  0.3065,  0.1448, -0.1141, -0.3625, -0.0264,  0.3279,  0.3900,
           0.1979, -0.1840,  0.1636,  0.1665,  0.3929, -0.1639, -0.1987, -0.2770]],
        grad_fn=<SliceBackward0>))

In [ ]:
# version 4: self-attention (Andrej version)

In [ ]:
torch.manual_seed(3)
B,T,C=4,8,32
x=torch.randn(B,T,C)

#single self-attention head
head_size=16
key=nn.Linear(C,head_size,bias=False)
query=nn.Linear(C,head_size,bias=False)
value=nn.Linear(C,head_size,bias=False)
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
v = value(x)

wei = q @ rearrange(k, 'B T C -> B C T')
tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3835, 0.6165, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1507, 0.2281, 0.6212, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0507, 0.0230, 0.7846, 0.1417, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2544, 0.2024, 0.4529, 0.0681, 0.0223, 0.0000, 0.0000, 0.0000],
        [0.1009, 0.5172, 0.1604, 0.1195, 0.0186, 0.0835, 0.0000, 0.0000],
        [0.1076, 0.0954, 0.0025, 0.0086, 0.7683, 0.0080, 0.0097, 0.0000],
        [0.2585, 0.0457, 0.0329, 0.0113, 0.0118, 0.3126, 0.2737, 0.0535]],
       grad_fn=<SelectBackward0>)

In [ ]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * (head_size**-0.5)
print(k.var(), q.var())
print(wei.var())

tensor(0.9504) tensor(1.0232)
tensor(0.9685)


In [ ]:
wei[0]

tensor([[-0.8225,  0.2513, -1.1796,  0.2286, -0.0963,  0.9684, -1.2620,  0.2212],
        [ 2.4607,  1.6196,  0.2774,  0.1037,  1.7921,  1.5087, -1.1270, -0.6625],
        [ 0.1954, -1.7517,  0.1640,  1.0055,  0.0459, -1.8845,  1.3148, -0.2771],
        [-2.0184, -0.1616, -0.1218, -0.1139,  1.0969,  0.4771, -0.4557, -0.4467],
        [-1.9438,  1.1667, -0.7137,  1.3427, -0.2675, -2.2083, -0.3305, -1.8594],
        [ 0.6739,  0.3095,  0.0882,  0.2718,  0.0699, -1.2526,  0.5267,  0.2534],
        [ 0.1975, -0.3789, -0.9172,  0.1914, -0.4382,  0.9326,  0.1505,  1.5192],
        [-0.2842,  1.0991, -0.0356, -0.6142, -0.7021,  0.4700, -1.3805,  0.2360]])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, 0.5]), dim=-1)

tensor([0.2245, 0.1663, 0.2742, 0.3349])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, 0.5])*8, dim=-1)

tensor([0.0327, 0.0030, 0.1620, 0.8023])

In [ ]:
class LayerNorm:

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.ones(dim)

  def __call__(self, x):
    xmean = x.mean(-1, keepdim=True)
    xvar = x.var(-1, keepdim=True)
    xhat = (x-xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]

module = LayerNorm(100)
x = torch.rand(32, 100)
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[0]

tensor([ 2.3455e+00,  2.6194e+00,  2.0827e+00,  1.3762e+00,  2.9404e-01,
        -1.2090e-02,  2.5746e-01,  7.3188e-01,  1.5219e+00, -1.8476e-01,
         7.3477e-01,  1.7242e+00,  2.5107e+00,  1.3145e+00,  1.2971e+00,
         5.0545e-01,  2.1720e+00,  8.3857e-01, -2.8253e-05,  2.4993e+00,
        -2.8490e-01, -5.5096e-01,  2.5085e+00,  1.6450e+00,  7.0065e-01,
         2.6866e-01,  1.4610e+00,  1.1637e+00, -5.9129e-01,  2.1673e+00,
         1.0234e+00,  2.5926e+00,  2.8687e-01, -2.4771e-01,  2.5922e+00,
         1.2024e+00,  1.7027e+00, -5.4927e-02,  1.3871e+00,  8.1647e-01,
         2.2686e+00,  1.9890e+00,  1.4315e+00,  1.3788e+00, -5.9318e-01,
         1.6901e+00, -4.0116e-01,  2.5742e+00,  2.2766e+00,  4.6099e-01,
         3.5855e-01,  2.0046e+00,  1.5602e+00,  9.3168e-01,  1.2217e+00,
         1.7688e+00,  1.6342e+00,  7.4533e-01,  1.0405e-01, -1.4352e-01,
        -6.6179e-02, -1.3787e-01, -4.2803e-01,  1.9904e+00,  2.4511e+00,
        -7.2737e-02, -5.1592e-01,  1.7863e-01,  4.4

In [ ]:
# residual connections are super important
# linearly project multi-head attention output, then dropout

#feed forward linear(n, 4n), GeLU, linear(4n, n), dropout

#pre norm formulation, creates gradient super highway!
#layer norm before it goes into self-attention and feedforward

#add layer norms after block before final linear layer

#scaling up module
#dropout after softmax

# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2